# Long-Short Strategy, Part 5: Generating out-of-sample predictions

In this section, we'll start designing, implementing, and evaluating a trading strategy for US equities driven by daily return forecasts produced by gradient boosting models.

As in the previous examples, we'll lay out a framework and build a specific example that you can adapt to run your own experiments. There are numerous aspects that you can vary, from the asset class and investment universe to more granular aspects like the features, holding period, or trading rules. See, for example, the **Alpha Factor Library** in the [Appendix](../24_alpha_factor_library) for numerous additional features.

We'll keep the trading strategy simple and only use a single ML signal; a real-life application will likely use multiple signals from different sources, such as complementary ML models trained on different datasets or with different lookahead or lookback periods. It would also use sophisticated risk management, from simple stop-loss to value-at-risk analysis.

**Six notebooks** cover our workflow sequence:

1. [preparing_the_model_data](04_preparing_the_model_data.ipyny): we engineer a few simple features from the Quandl Wiki data 
2. [trading_signals_with_lightgbm_and_catboost](05_trading_signals_with_lightgbm_and_catboost.ipynb): we tune hyperparameters for LightGBM and CatBoost to select a model, using 2015/16 as our validation period. 
3. [evaluate_trading_signals](06_evaluate_trading_signals): we compare the cross-validation performance using various metrics to select the best model. 
4. [model_interpretation](07_model_interpretation.ipynb): we take a closer look at the drivers behind the best model's predictions.
5. `making_out_of_sample_predictions` (this noteboook): we predict returns for our out-of-sample period 2017.
6. [backtesting_with_zipline](09_backtesting_with_zipline.ipynb): evaluate the historical performance of a long-short strategy based on our predictive signals using Zipline.

## Imports & Settings

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

from time import time
import sys, os
from pathlib import Path

import pandas as pd
from scipy.stats import spearmanr

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import MultipleTimeSeriesCV

In [4]:
sns.set_style('whitegrid')

In [5]:
YEAR = 252
idx = pd.IndexSlice

In [6]:
scope_params = ['lookahead', 'train_length', 'test_length']
daily_ic_metrics = ['daily_ic_mean', 'daily_ic_mean_n', 'daily_ic_median', 'daily_ic_median_n']
lgb_train_params = ['learning_rate', 'num_leaves', 'feature_fraction', 'min_data_in_leaf']
catboost_train_params = ['max_depth', 'min_child_samples']

## Generate LightGBM predictions

### Model Configuration

In [59]:
base_params = dict(boosting='gbdt',
                   objective='regression',
                   verbose=-1)

categoricals = ['year', 'month', 'sector', 'weekday']

In [60]:
lookahead = 1
store = Path('data/predictions.h5')

### Get Data

In [61]:
data = pd.read_hdf('data.h5', 'model_data').sort_index()

In [62]:
labels = sorted(data.filter(like='_fwd').columns)
features = data.columns.difference(labels).tolist()
label = f'r{lookahead:02}_fwd'

In [63]:
data = data.loc[idx[:, '2000':], features + [label]].dropna()

In [64]:
for feature in categoricals:
    data[feature] = pd.factorize(data[feature], sort=True)[0]

In [65]:
lgb_data = lgb.Dataset(data=data[features],
                       label=data[label],
                       categorical_feature=categoricals,
                       free_raw_data=False)

### Generate predictions

In [7]:
lgb_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/ic')
lgb_daily_ic = pd.read_hdf('data/model_tuning.h5', 'lgb/daily_ic')

In [8]:
def get_lgb_params(data, t=5, best=0):
    param_cols = scope_params[1:] + lgb_train_params + ['boost_rounds']
    df = data[data.lookahead==t].sort_values('ic', ascending=False).iloc[best]
    return df.loc[param_cols]

In [68]:
for position in range(10):
    params = get_lgb_params(lgb_daily_ic,
                            t=lookahead,
                            best=position)

    params = params.to_dict()

    for p in ['min_data_in_leaf', 'num_leaves']:
        params[p] = int(params[p])
    train_length = int(params.pop('train_length'))
    test_length = int(params.pop('test_length'))
    num_boost_round = int(params.pop('boost_rounds'))
    params.update(base_params)

    print(f'\nPosition: {position:02}')

    # 1-year out-of-sample period
    n_splits = int(YEAR / test_length)
    cv = MultipleTimeSeriesCV(n_splits=n_splits,
                              test_period_length=test_length,
                              lookahead=lookahead,
                              train_period_length=train_length)

    predictions = []
    start = time()
    for i, (train_idx, test_idx) in enumerate(cv.split(X=data), 1):
        print(i, end=' ', flush=True)
        lgb_train = lgb_data.subset(used_indices=train_idx.tolist(),
                                    params=params).construct()

        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=num_boost_round)

        test_set = data.iloc[test_idx, :]
        y_test = test_set.loc[:, label].to_frame('y_test')
        y_pred = model.predict(test_set.loc[:, model.feature_name()])
        predictions.append(y_test.assign(prediction=y_pred))

    if position == 0:
        test_predictions = (pd.concat(predictions)
                            .rename(columns={'prediction': position}))
    else:
        test_predictions[position] = pd.concat(predictions).prediction

by_day = test_predictions.groupby(level='date')
for position in range(10):
    if position == 0:
        ic_by_day = by_day.apply(lambda x: spearmanr(
            x.y_test, x[position])[0]).to_frame()
    else:
        ic_by_day[position] = by_day.apply(
            lambda x: spearmanr(x.y_test, x[position])[0])
print(ic_by_day.describe())
test_predictions.to_hdf(store, f'lgb/test/{lookahead:02}')


Position: 00
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 01
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 02
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 03
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 04
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 05
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 06
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 07
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 08
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4 
Position: 09
---------{dates}
1 ---------{dates}
2 ---------{dates}
3 ---------{dates}
4                 0           1           2           3           4           5  \
count  252.000000  

## Generate CatBoost predictions

In [69]:
# lookaheads = [1, 5, 21]
lookaheads = [1]

In [70]:
label_dict = dict(zip(lookaheads, labels))

### Model Configuration

In [71]:
lookahead = 1
store = Path('data/predictions.h5')

### Get Data

In [72]:
data = pd.read_hdf('data.h5', 'model_data').sort_index()

In [73]:
labels = sorted(data.filter(like='_fwd').columns)
features = data.columns.difference(labels).tolist()
label = f'r{lookahead:02}_fwd'

In [74]:
data = data.loc[idx[:, '2000':], features + [label]].dropna()

In [75]:
for feature in categoricals:
    data[feature] = pd.factorize(data[feature], sort=True)[0]

In [76]:
cat_cols_idx = [data.columns.get_loc(c) for c in categoricals]

In [77]:
catboost_data = Pool(label=data[label],
                     data=data.drop(label, axis=1),
                     cat_features=cat_cols_idx)

### Generate predictions

In [78]:
catboost_ic = pd.read_hdf('data/model_tuning.h5', 'catboost/ic')
catboost_ic_avg = pd.read_hdf('data/model_tuning.h5', 'catboost/daily_ic')

In [79]:
def get_cb_params(data, t=5, best=0):
    param_cols = scope_params[1:] + catboost_train_params + ['boost_rounds']
    df = data[data.lookahead==t].sort_values('ic', ascending=False).iloc[best]
    return df.loc[param_cols]

In [80]:
for position in range(10):
    params = get_cb_params(catboost_ic_avg,
                    t=lookahead,
                    best=position)
    
    params = params.to_dict()
    
    for p in ['max_depth', 'min_child_samples']:
        params[p] = int(params[p])
    train_length = int(params.pop('train_length'))
    test_length = int(params.pop('test_length'))
    num_boost_round = int(params.pop('boost_rounds'))
    # params['task_type'] = 'GPU'

    print(f'\nPosition: {position:02}')
    
    # 1-year out-of-sample period
    n_splits = int(YEAR / test_length)
    cv = MultipleTimeSeriesCV(n_splits=n_splits,
                              test_period_length=test_length,
                              lookahead=lookahead,
                              train_period_length=train_length)

    predictions = []
    start = time()
    for i, (train_idx, test_idx) in enumerate(cv.split(X=data), 1):
        print(i, end=' ', flush=True)
        train_set = catboost_data.slice(train_idx.tolist())

        model = CatBoostRegressor(**params)
        model.fit(X=train_set)

        test_set = data.iloc[test_idx, :]
        y_test = test_set.loc[:, label].to_frame('y_test')
        y_pred = model.predict(test_set.loc[:, model.feature_names_])
        predictions.append(y_test.assign(prediction=y_pred))

    if position == 0:
        test_predictions = (pd.concat(predictions)
                            .rename(columns={'prediction': position}))
    else:
        test_predictions[position] = pd.concat(predictions).prediction

by_day = test_predictions.groupby(level='date')
for position in range(10):
    if position == 0:
        ic_by_day = by_day.apply(lambda x: spearmanr(x.y_test, x[position])[0]).to_frame()
    else:
        ic_by_day[position] = by_day.apply(lambda x: spearmanr(x.y_test, x[position])[0])
print(ic_by_day.describe())
test_predictions.to_hdf(store, f'catboost/test/{lookahead:02}')


Position: 00
---------{dates}
1 Learning rate set to 0.074502
0:	learn: 0.0331335	total: 9.19ms	remaining: 9.18s
1:	learn: 0.0331045	total: 17ms	remaining: 8.49s
2:	learn: 0.0330856	total: 23.2ms	remaining: 7.73s
3:	learn: 0.0330643	total: 29.4ms	remaining: 7.33s
4:	learn: 0.0330454	total: 34.1ms	remaining: 6.78s
5:	learn: 0.0330311	total: 38.9ms	remaining: 6.45s
6:	learn: 0.0330125	total: 43.6ms	remaining: 6.18s
7:	learn: 0.0329942	total: 48.4ms	remaining: 6.01s
8:	learn: 0.0329340	total: 53.8ms	remaining: 5.92s
9:	learn: 0.0328778	total: 57.3ms	remaining: 5.67s
10:	learn: 0.0328488	total: 60.7ms	remaining: 5.46s
11:	learn: 0.0328063	total: 64ms	remaining: 5.27s
12:	learn: 0.0327934	total: 68.9ms	remaining: 5.23s
13:	learn: 0.0327821	total: 73.5ms	remaining: 5.18s
14:	learn: 0.0327673	total: 78.3ms	remaining: 5.14s
15:	learn: 0.0327327	total: 81.6ms	remaining: 5.02s
16:	learn: 0.0327187	total: 86.5ms	remaining: 5s
17:	learn: 0.0326919	total: 89.4ms	remaining: 4.88s
18:	learn: 0.03267